In [8]:
import os
import shutil
import random

# Define source and destination paths
dataset_path = r"C:\Face Mask\data"
train_path = r"C:\Face Mask\data\train"
val_path = r"C:\Face Mask\data\val"

# Define categories
categories = ["with_mask", "without_mask"]

# Split ratio
split_ratio = 0.8  # 80% train, 20% validation

# Create train and val directories
for category in categories:
    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    os.makedirs(os.path.join(val_path, category), exist_ok=True)

    # Get all images in category folder
    images = os.listdir(os.path.join(dataset_path, category))
    random.shuffle(images)

    # Split data
    train_size = int(len(images) * split_ratio)
    train_images = images[:train_size]
    val_images = images[train_size:]

    # Move images to train and val folders
    for img in train_images:
        shutil.move(os.path.join(dataset_path, category, img), os.path.join(train_path, category, img))

    for img in val_images:
        shutil.move(os.path.join(dataset_path, category, img), os.path.join(val_path, category, img))

print("✅ Dataset successfully split into train and validation sets!")

✅ Dataset successfully split into train and validation sets!


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = r"C:\Face Mask\data\train"
val_dir = r"C:\Face Mask\data\val"

# Image Data Generators (Augmentation)
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load images
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode="binary")
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, class_mode="binary")

# Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (Cats vs. Dogs)
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save model
model.save("face_mask_classifier.h5")
print("✅ Model training complete and saved!")

Found 5996 images belonging to 2 classes.
Found 1500 images belonging to 2 classes.


c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 31/188 ━━━━━━━━━━━━━━━━━━━━ 1:21 522ms/step - accuracy: 0.5385 - loss: 0.8974

c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


188/188 ━━━━━━━━━━━━━━━━━━━━ 107s 566ms/step - accuracy: 0.7448 - loss: 0.5193 - val_accuracy: 0.8980 - val_loss: 0.2507
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 71s 378ms/step - accuracy: 0.8989 - loss: 0.2551 - val_accuracy: 0.9007 - val_loss: 0.2496
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 73s 387ms/step - accuracy: 0.9215 - loss: 0.2039 - val_accuracy: 0.9067 - val_loss: 0.2691
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 71s 378ms/step - accuracy: 0.9280 - loss: 0.1893 - val_accuracy: 0.9340 - val_loss: 0.1802
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 73s 385ms/step - accuracy: 0.9463 - loss: 0.1455 - val_accuracy: 0.9340 - val_loss: 0.1903
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 76s 402ms/step - accuracy: 0.9472 - loss: 0.1474 - val_accuracy: 0.9460 - val_loss: 0.1577
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 73s 390ms/step - accuracy: 0.9517 - loss: 0.1289 - val_accuracy: 0.9473 - val_loss: 0.1737
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 72s 381ms/step - accuracy: 0.9574 - loss: 0.1230 - va

✅ Model training complete and saved!


In [21]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load trained model
model = tf.keras.models.load_model("face_mask_classifier.h5")

# Load and preprocess test image
img_path = r"C:\Users\Pc\Downloads\smile.jpeg"
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)
prediction

if prediction > 0.5:
   print("Without Mask")
else:
   print("With Mask")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Without Mask
